In [ ]:
import sys
sys.path.append("../helper_functions")
import defillama_utils as dfl
sys.path.pop()
import requests as r
import pandas as pd
import plotly.express as px

chain = 'Base'

In [ ]:
# dt = r.get('https://api.llama.fi/protocol/aave-v3').json()
# dt = dt['chainTvls']['Optimism']['tokens']
# dt = pd.json_normalize(dt)
# print(dt)

In [ ]:
df_base = dfl.get_protocols_by_chain(chain)
df_base = df_base[['category','name','slug']]
df_base['chain'] = chain

In [ ]:
tvl_dfs = []
for i, row in df_base.iterrows():
        try:
                df = dfl.get_single_tvl(row['slug'],[row['chain']])
                df = df.groupby(['date','chain','protocol','category']).agg({'total_app_tvl':'mean'})
                df.reset_index(inplace=True)
                df['name'] = row['name']
                tvl_dfs.append(df)
        except:
                print('error: ' + row['name'])
                continue

tvl_df = pd.concat(tvl_dfs)
tvl_df = tvl_df[tvl_df['total_app_tvl'] > 0]
tvl_df['date'] = pd.to_datetime(tvl_df['date'])
display(tvl_df)

In [ ]:
launch_df = tvl_df.groupby(['chain','protocol','category','name']).agg({'date':'min', 'total_app_tvl':'last'})
launch_df = launch_df.rename(columns={
        'date':'launch_date',
        'total_app_tvl':'latest_tvl_usd'
})
launch_df.reset_index(inplace=True)
display(launch_df)

In [17]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Load your data into a pandas DataFrame
df = launch_df.copy()

# Create a subplot with 1 row and 2 columns, where the second is a bar chart that shares the x-axis
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scatter"}, {"type": "bar"}]], shared_xaxes=True, 
                    column_widths=[0.7, 0.3])

# Scatter plot for TVL and launch date
scatter = go.Scatter(
    x=df['launch_date'],
    y=df['latest_tvl_usd'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df['latest_tvl_usd']/1e6),  # Size of markers can be scaled down for better visualization
        color=df['category'].astype('category').cat.codes, # Convert category to numeric codes for color
        showscale=True  # Shows color scale beside
    ),
    text=df['name'],  # Will show up on hover
    name='TVL'
)

# Add scatter plot to the first column
fig.add_trace(scatter, row=1, col=1)

# Group by week and count the number of launches
df['week'] = df['launch_date'].dt.to_period('W').apply(lambda r: r.start_time)
weekly_launch_counts = df.groupby('week')['name'].count().reset_index()

# Bar chart for the number of launches per day
# launch_counts = df['launch_date'].value_counts().sort_index()
bar = go.Bar(
    x=weekly_launch_counts['week'],
    y=weekly_launch_counts['name'],
    name='Launches Count'
)

# Add bar chart to the second column
fig.add_trace(bar, row=1, col=2)

# Set plot titles and labels
fig.update_layout(title_text='Project Impact and Launch Timeline with Launch Counts',
                  xaxis_title='Launch Date',
                  yaxis_title='Latest TVL (USD)',
                  bargap=0.2)

# Set y-axis to logarithmic if needed (it's been requested as non-logarithmic, but it's here if necessary)
# fig.update_yaxes(type="log", row=1, col=1)

# Update xaxis properties for the bar chart to prevent label overlap
fig.update_xaxes(tickangle=45, tickfont=dict(size=10), row=1, col=2)

# Update layout to adjust the space to prevent the labels from being cut off
fig.update_layout(margin=dict(l=20, r=20, t=85, b=20))

# Show the plot
fig.show()
